In [160]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [161]:
data = pd.read_csv("spec.csv")

In [162]:
# '자격증' feature 삭제
data.drop(['평균 토익스피킹 점수','평균 오픽점수', '자격증'], axis=1, inplace=True)

In [163]:
data['기업명'] = data['기업명'].str.replace('㈜', '', regex=False)
data['기업명'] = data['기업명'].str.replace('(주)', '', regex=False)
data['기업명'] = data['기업명'].str.replace('(유)', '', regex=False)

In [164]:
# '-' 문자를 NaN으로 변경
data.replace('-', np.nan, inplace=True)

In [165]:
# '평균 토익점수' 컬럼을 수치형으로 변환
data['평균 토익점수'] = data['평균 토익점수'].apply(lambda x: float(x.replace('점', '')) if isinstance(x, str) else x)

In [166]:
# '평균 외국어 개수', '평균 자격증 개수' 컬럼을 수치형으로 변환
for column in ['평균 외국어 개수', '평균 자격증 개수']:
    data[column] = data[column].apply(lambda x: float(x.replace('개', '')) if isinstance(x, str) else x)

In [167]:
# '평균 해외경험 횟수', '평균 인턴경험 횟수', '평균 수상 횟수', '평균 봉사 횟수' 컬럼을 수치형으로 변환
for column in ['평균 해외경험 횟수', '평균 인턴경험 횟수', '평균 수상 횟수', '평균 봉사 횟수']:
    data[column] = data[column].apply(lambda x: float(x.replace('회', '')) if isinstance(x, str) else x)

In [168]:
# 각 특성에 대해 반복
for column in data.columns:
    # '기업명'과 '평균 학점' 열은 제외
    if column in ['평균 토익점수', '평균 외국어 개수', '평균 해외경험 횟수', '평균 인턴경험 횟수', '평균 수상 횟수', '평균 봉사 횟수']:
        # 결측값이 없는 행과 결측값이 있는 행 분리
        train_data = data[data[column].notna()]
        test_data = data[data[column].isna()]

        # 훈련 데이터 준비
        X_train = train_data['평균 학점'].values.reshape(-1, 1)
        y_train = train_data[column]

        # 모델 학습
        model = LinearRegression()
        model.fit(X_train, y_train)

        # 결측값 예측
        X_test = test_data['평균 학점'].values.reshape(-1, 1)
        predicted_values = model.predict(X_test)

        # 예측된 값으로 결측값 채우기
        data.loc[data[column].isna(), column] = predicted_values

In [169]:
print(data)

              기업명  평균 학점    평균 토익점수  평균 외국어 개수  평균 자격증 개수  평균 해외경험 횟수  \
0            삼성전자   3.72  833.00000   1.200000        2.2         1.3   
1            우리은행   3.64  838.00000   1.100000        2.2         1.2   
2            신한은행   3.72  841.00000   1.000000        2.4         1.1   
3     한국스탠다드차타드은행   3.55  843.00000   1.100000        2.0         1.2   
4          한국전력공사   3.65  851.00000   1.200000        2.4         1.4   
...           ...    ...        ...        ...        ...         ...   
3016        에코이에스   3.56  600.00000   1.157136        2.8         1.0   
3017         농협홍삼   3.31  757.00000   1.000000        1.6         1.5   
3018        포스하이알   3.73  789.66191   2.500000        2.2         1.0   
3019      현대커민스엔진   3.68  816.00000   1.000000        2.8         1.0   
3020       원신스카이텍   3.66  690.00000   1.500000        2.1         1.3   

      평균 인턴경험 횟수  평균 수상 횟수  평균 봉사 횟수  
0       1.300000  2.200000  2.000000  
1       1.300000  1.700000  1.600000  
2     

In [171]:
# 새로운 CSV 파일로 저장
data.to_csv("preprocessed_spec.csv", index=False)